# Web Scraping with a Programmatic Browser

Many website use a lot of javascript to render the navigation more dynamic and optimize display speed by only loading the content that can be displayed on the screen of the user devices.

For these websites, writing a scraper at the HTTP + HTML level is too complex: one would need to record and analyze all the HTTP request / responses of a typical navigation session to understand how to simulate it with low level tools such as `urllib` and `lxml`.

An alternative is to programmatically control the execution of a full-fledged browser like Firefox, Chrome, Edge or Safari and introspect the state of the Document object model. This way it is possible to execute all the javascript necessary to trigger navigation events by using an API that reflects how the end user would interact with the web page (clicking on links, mouse scroll events, pressing keys on the keyboard to type text in forms and so on).

To illustrate this approach, let's write a program that simulates the navigation to the Book section of the amazon online shop and retrieve the list of the title of best sellers along with their respective price.

To do this exercises you will need to install the Python package for `selenium` (that includes the `webdriver` API), the [Firefox](https://www.mozilla.org/en-US/firefox/) web browser and the [geckodriver](https://github.com/mozilla/geckodriver/releases) helper program (de-zip it and put it somewhere in your PATH, for instance in the `bin/` or `Scripts/` folder of your conda environment.

In [ ]:
import sys
!{sys.executable} -m pip install selenium

In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Firefox()

In [ ]:
driver.get("https://www.amazon.com")

## Navigation to the Book Page

In [ ]:
driver.find_elements_by_partial_link_text("book")

In [ ]:
driver.find_elements_by_link_text("Departments")

In [ ]:
departments_link = driver.find_element_by_link_text("Departments")

In [ ]:
departments_link.click()

In [ ]:
driver.find_elements_by_link_text("Books")

In [ ]:
driver.find_element_by_link_text("Books").click()

## Finding the List of Best Sellers

In [ ]:
carousels = driver.find_elements_by_class_name("acswidget-carousel")
len(carousels)

In [ ]:
carousels = driver.find_elements_by_css_selector(".acswidget-carousel")
len(carousels)

In [ ]:
first_carousel = carousels[0]
first_carousel.tag_name

In [ ]:
print(first_carousel.text)

In [ ]:
for i, carousel in enumerate(carousels):
    print(f"Text content of carousel #{i}:")
    print(carousel.text[:100])

In [ ]:
bestsellers_carousels = [carousel for carousel in carousels
                         if "Books Bestsellers" in carousel.text]
len(bestsellers_carousels)

In [ ]:
[bestsellers_carousel] = bestsellers_carousels

In [ ]:
nextpage_button = bestsellers_carousel.find_element_by_class_name("a-carousel-goto-nextpage")

In [ ]:
nextpage_button.click()

In [ ]:
driver.execute_script("arguments[0].click();", nextpage_button)

In [ ]:
cards = bestsellers_carousel.find_elements_by_class_name("a-carousel-card")
len(cards)

Try to resize the firefox window or reduce the font size and fetch again the list of cards in the `bestsellers_carousel` element.

In [ ]:
cards = bestsellers_carousel.find_elements_by_class_name("a-carousel-card")
len(cards)

Notice that the content can change dynamically: the javascript in the page reacts to font-size or window-size change events to load the information of more books and update the DOM dynamically.

Let's consider the first "carousel card" element of our updated carousel.

In [ ]:
first_card = next(iter(cards))

In [ ]:
print(first_card.text)

In [ ]:
for link in first_card.find_elements_by_tag_name("a"):
    print(link.text)
    print(f"  {link.get_attribute("href")}")

In [ ]:
product_links = [tag for tag in first_card.find_elements_by_tag_name("a")
                 if "/product/" in tag.get_attribute("href")]
product_links

In [ ]:
[link.text for link in product_links]

## Exercises:

- Use Firefox "Inspect Element" tool to analyse the DOM and find a way to extract the element that contains the name of the book, its product id and its the price element for `first_card`;

- Write a function named `extract_bookname(card)` that extracts the name of the book described in a "carousel card" element. Try your function on `first_card`;

- Write a function named `extract_price(card)` that extracts the **numerical** price for the book described in a "carousel card" element. Try the function on `first_card`;

- Write a function named `extract_product_id(card)` that extracts the Amazon product identifier from the URL of the product link in the card.

In [ ]:
def extract_bookname(book_card):
    # TODO: implement me
    return "The story of my life"


extract_bookname(first_card)

In [ ]:
# %load notebook_solutions/extract_bookname.py

In [ ]:
def extract_price(book_card):
    # TODO: implement me!
    return 10.0


extract_price(first_card)

In [ ]:
# %load notebook_solutions/extract_price.py

In [ ]:
def extract_product_id(book_card):
    # TODO: implement me!
    return 238834593244


extract_product_id(first_card)

In [ ]:
# %load notebook_solutions/extract_product_id.py

Once we no longer need the browser, we can close the window to free system resources:

In [ ]:
driver.close()

## Exercise

Write a function that takes that creates its own driver, navigates to the book departments and retrieve the top 30 book sellers. For each book extract the amazon product identifier, the title of the book and the price.

Return the results in a list of Python dictionaries or a pandas dataframe.

In [ ]:
# %load notebook_solutions/scrape_best_sellers.py

## Using Firefox in Headless Mode

Being able to view the firefox window is nice when developing, for instance to use the firefox developer tools to introspect the structure of the DOM. However once the script is ready, have to use the graphical interface can use additional resources for nothing. In that case, it's possible to pass a specific environment variable to firefox to run in headless mode. Alternatively one could use a naturally headless browser such as phantomjs instead of Firefox.

In [ ]:
import os

try:
    os.environ['MOZ_HEADLESS'] = '1'
    driver = webdriver.Firefox()
    driver.get("https://www.scikit-learn.org/stable")
    
    for title in driver.find_elements_by_tag_name("h2"):
        print(title.text)
        
finally:
    driver.close()
    del os.environ['MOZ_HEADLESS']